Phase 1 - Scraping facebook posts.

The following code will scrape facebook posts from a public facebook group given the following parameters:
access_token - a facebook developers access token which is valid for 2 hours from the moment it is generated

you can get your own access token by following the instructions on: from https://developers.facebook.com/docs/facebook-login/access-tokens

**NOTE: THE ACCESS TOKEN HARDCODED HERE IS NO LONGER VALID FBY THE TIME YOU READ THIS, SO IN ORDER TO RUN THE PROJECT YOU MUST CREATE YOUR OWN ACCESS TOKEN**

**NOTE: RUN THIS CODE WITH PYTHON 2 ONLY OR IT WON'T WORK**

This box of code creates a .csv file which contains the posts scraped from the given group.

In [ ]:
import urllib2
import json
import datetime
import csv
import time

#change the group ID to another group if you want to, currently it will scrape the group with the id 143136595007
group_id = "143136595007"

#THIS CODE IS FOR SCRAPING POSTS FROM A PUBLIC FACEBOOK GROUP GIVEN THE GROUP ID AND AN ACCESS TOKEN.
#THE ACCESS TOKEN IS TEMPORARY AND EXPIRES AFTER 2 HOURS.

#Enter your facebook developer's access token. Note: it will stop working after 2 hours, so make sure that after 2 hours you paste in 
#the new access token

access_token = "EAACEdEose0cBAO2UyNmch9icbTO9UpIV4gZBOBCYrCrPs7xBUlJO3CMNUJuZCDU5XT2HJmzYW8pKZCCKF8DRZCOFQjeiJZAEcLZCone7ZBQQmSV7Ia1TAtiKpvYZBivDIZBzsOCRZAN2i3eKrT6uKR3nSf8n0eBZC2uaJ5PjrjIzPlTPJUVt8I0qrmZBbvJFMTLZCfmAZD"


def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            print "Error for URL %s: %s" % (url, datetime.datetime.now())
            print "Retrying."

    return response.read()

# Needed to write tricky unicode correctly to csv
def unicode_normalize(text):
    return text.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22,
                            0xa0:0x20 }).encode('utf-8')

def getFacebookPageFeedData(group_id, access_token, num_statuses):
    # Construct the URL string; see 
    # http://stackoverflow.com/a/37239851 for Reactions parameters
    base = "https://graph.facebook.com/v2.6"
    node = "/%s/feed" % group_id 
    fields = "/?fields=message,link,permalink_url,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions." + \
            "limit(0).summary(true),from"
    parameters = "&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + fields + parameters

    # retrieve data
    data = json.loads(request_until_succeed(url))

    return data

def processFacebookPageFeedStatus(status, access_token):

    # We decided to make it so that the status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first

    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else \
            unicode_normalize(status['message'])
    link_name = '' if 'name' not in status.keys() else \
            unicode_normalize(status['name'])
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else \
            unicode_normalize(status['link'])
    status_permalink_url = '' if 'permalink_url' not in status.keys() else \
            unicode_normalize(status['permalink_url'])
    status_author = unicode_normalize(status['from']['name'])
    status_published = datetime.datetime.strptime(\
            status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    # best time format for spreadsheet programs:
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S')

    return (status_id, status_message, status_author, link_name, status_type, 
            status_link, status_permalink_url, status_published)

def scrapeFacebookPageFeedStatus(group_id, access_token):
    with open('facebook_statuses.csv', 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "status_author", 
            "link_name", "status_type", "status_link","permalink_url",
            "status_published"])

        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()

        print "Scraping %s Facebook Page: %s\n" % \
                (group_id, scrape_starttime)

        statuses = getFacebookPageFeedData(group_id, access_token, 100)

        while has_next_page:
            for status in statuses['data']:
                if 'reactions' in status:            
                    w.writerow(processFacebookPageFeedStatus(status, \
                                  access_token))
                num_processed += 1
                if num_processed % 100 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, 
                            datetime.datetime.now())
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(\
                        statuses['paging']['next']))
            #dont scrape more than 6000 posts because it takes more than 2 hours and the access token will expire
            elif num_processed == 6000:
                has_next_page = False
            else:
                has_next_page = False

        print "\nDone!\n%s Statuses Processed in %s" % \
                (num_processed, datetime.datetime.now() - scrape_starttime)


if __name__ == '__main__':
    scrapeFacebookPageFeedStatus(group_id, access_token)


PHASE 1 (cont.) 
After we have the scraped posts in .csv files for the group/groups we want to subset the data and clear out unwanted posts. We also want to classify and separate male an female posts to 2 different .csv files: 1 for male posts and 1 for female posts. 

RUN THIS CODE WITH **PYTHON 3** ONLY OR IT WON'T WORK

Some notes regarding the data gathering part:
Facebook doesn't allow scraping people's personal statuses even if they are posted as 'public', but it does allow scraping
posts from GROUPS that are public. Unfortunately, this would mean that posts will not be more generic and probably correlate to the author's gender as the post is less likely to contain a 'personal tone' and focus more on the topic of the group. That means that the posts are less likely to truly represent male or female behavior because the posts will be more in the context of the group topic or a serious discussion. 

We decided to scrape posts from 2 different groups, both of which tech-oriented. The 1st group is called 'Big Data, Data Science, Data Mining & Statistics'
and the second group is called 'Science, Technology, and Society Discussion Corner'.
Naturally, that would mean the posts would likely focus on tech and data.

Another big problem is that Facebook doesn't allow us to have the gender of the post author, but will try to solve this problem later on.
See here: https://stackoverflow.com/questions/24945763/facebook-graph-api-how-to-get-gender-from-public-profile


In [2]:
import pandas as pd
#After scraping the posts from the 2 groups and inserting them into a .csv files, we read files and create a dataframe.
facebook_posts_group1 = pd.DataFrame.from_csv('143136595007_facebook_statuses.csv')
facebook_posts_group2 = pd.DataFrame.from_csv('484618291590939_facebook_statuses.csv')

Combine the 2 files into a single dataframe:

In [3]:
facebook_posts_selected_columns_group1 = facebook_posts_group1[['status_message','status_author', 'status_type']]
facebook_posts_selected_columns_group2 = facebook_posts_group2[['status_message','status_author', 'status_type']]

facebook_groups_posts = pd.concat([facebook_posts_selected_columns_group1, facebook_posts_selected_columns_group2], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)

facebook_groups_posts.head(10)

,status_message,status_author,status_type
status_id,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status
484618291590939_2240758702643547,NaN,Foued Saâdaoui,link
484618291590939_2239120939473990,If you want to solve some real-world problems ...,Matt Mayo,link
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status
484618291590939_2238306732888744,NaN,Saimadhu Polamuri,link
484618291590939_2238261742893243,----------------------\nCall for Book Chapters...,Kuan-Ching Li,status
484618291590939_2237867916265959,NaN,Joseph Anoop George,status


Next:
CLEANING THE DATASET

remove statuses that are too short (less than 2 words) because they are more likely to be spam or meaningless.
Then remove rows containing these spam messages.

In [12]:
import numpy as np
posts_empties_removed = facebook_groups_posts.dropna(axis=0, how='any')

for index, row in posts_empties_removed.iterrows():
    if (len(row['status_message'].split(" ")) == 1):
        row['status_message'] = np.nan

posts_empties_removed2 = posts_empties_removed.dropna(axis=0, how='any')
posts_empties_removed2.head(10)

,status_message,status_author,status_type
status_id,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status
484618291590939_2239120939473990,If you want to solve some real-world problems ...,Matt Mayo,link
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status
484618291590939_2238261742893243,----------------------\nCall for Book Chapters...,Kuan-Ching Li,status
484618291590939_2237858432933574,This post will take a different approach to co...,Matt Mayo,link
484618291590939_2237567572962660,#Reinforcementlearning to play an important ro...,Bharath Reddy,link
484618291590939_2236526149733469,"Riskified is hiring ! \nAs a Big Data expert, ...",Maya Salemnia,link


Throw away posts that are nothing but a link or a video description, because they do not contain text that was written by the author, it only contains the description of the link/video they posted.

In [5]:
posts_statuses_only = posts_empties_removed2[(posts_empties_removed2.status_type != 'link') & (posts_empties_removed2.status_type != 'video')]
posts_statuses_only.head(10)

,status_message,status_author,status_type
status_id,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status
484618291590939_2238261742893243,----------------------\nCall for Book Chapters...,Kuan-Ching Li,status
484618291590939_2236102949775789,Tweet Wars! Do you remember our sentiment anal...,Jamie Jones,photo
484618291590939_2232146760171408,Join the Big Data 5 Month Course + Project \nH...,Simran Kaur,photo
484618291590939_2230118397040911,If you are interested please drop me an email ...,Ritin Patel,photo
484618291590939_2224717017581049,"Most #banking, #financial services, and #insur...",Thomas Crystal,status


NEXT - DETERMINING THE GENDER OF USERS

Since Facebook won't allow us information such as the gender of the post author, we will have to turn to another solution. 
We found a python package that can guess the gender of a person judging by their first name using a dataset created from machine learning 
The package will determine if a name belongs to a male or female with 90+ accuracy. With 80+ accuracy, it will classify as mostly_male/mostly_female
If the accuracy is lower than 80%, it will classify as 'unknown'.
For the sake of accuracy, we will only keep those classified as male or female with 90+ accuracy.

First, we need to add a 'gender' column and a 'first_name' column, and then populate the gender column with the gender detector result.
Afterwards, we need to clear out those who were classified as 'mostly X' or 'unknown'.

In [6]:
 import gender_guesser.detector as gender

detector = gender.Detector()

lst = []
iterr = 0
posts_statuses_only['first_name'] = ""

#split the full names to first and last names
for index, row in posts_statuses_only.iterrows():
    lst.append(row['status_author'].split(" "))

#populate the first name coulmn with the first names from the splits
for index, row in posts_statuses_only.iterrows():
    row['first_name'] = lst[iterr][0]
    iterr = iterr+1

#populate the gender column with the class prediction using the gender guessing package
posts_statuses_only['gender'] = ""
iterr=0
for index, row in posts_statuses_only.iterrows():
    row['gender'] = detector.get_gender(row['first_name'])
    iterr = iterr+1


C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Separate male and female posts to different .csv files.

In [7]:
#only the male gender part of the dataframe
male_posts = posts_statuses_only[posts_statuses_only["gender"] == "male"]
#only the female gender part of the dataframe
female_posts = posts_statuses_only[posts_statuses_only["gender"] == "female"]

In [8]:
#This is a dataframe on male posts
male_posts.head(10)

,status_message,status_author,status_type,first_name,gender
status_id,,,,,
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status,Aditya,male
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status,Julian,male
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status,Badr,male
484618291590939_2224717017581049,"Most #banking, #financial services, and #insur...",Thomas Crystal,status,Thomas,male
484618291590939_2218490481537036,Urgent Hiring for the following job SEO / SOCI...,Michael Christian Veras,photo,Michael,male
484618291590939_2207498585969559,"Hi, I'm searching a subject about combined int...",Badr Rdab,status,Badr,male
484618291590939_2203750419677709,I'm searching for case studies about Data Mini...,Carmelo Saffioti,status,Carmelo,male
484618291590939_2190444491008302,HAHAHA\nBitcoin Vs Money by Google Trend!\nBit...,Muhammad Fahad,status,Muhammad,male
484618291590939_2188021981250553,Walk-in for Data Engineering Professionals\n\n...,Mahesh Babu Doraivelu,event,Mahesh,male


In [10]:
#This is a dataframe of female posts
female_posts.head(10)

,status_message,status_author,status_type,first_name,gender
status_id,,,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status,Juana,female
484618291590939_2164214636964621,#BLACKFRIDAYDEAL This is the best offer I foun...,Martha Grace,photo,Martha,female
484618291590939_2149002608485824,"Hello Hyderabad,\nNow IIMSE Started BIGDATA HA...",Sandhya Chowdary,photo,Sandhya,female
484618291590939_2143841505668601,Hi All. Have anyone considered doing a researc...,Bala Sekar,status,Bala,female
484618291590939_2143552672364151,Don't forget to like our Facebook page 👍\n▶ ht...,Stella Morvan,photo,Stella,female
484618291590939_2143372895715462,Join our 48 hour hackathon on a moving train s...,Sarah Chok,event,Sarah,female
484618291590939_2137515939634491,Transform your organization with powerful anal...,Emily At Qlik,photo,Emily,female
484618291590939_2134485076604244,"Hello Guys,\n\nIf anybody is interested to pro...",Divya Tiwari,status,Divya,female
484618291590939_2114258738626878,Visit Spotinst at Serverlessconf NYC\nSpotinst...,Anat Kaplinski,photo,Anat,female


Calculate some basic statistics about out data:

In [30]:
#Statistics about our data:

#Let's calculate the males' average number of words in a post
posts_list_male = male_posts.status_message.tolist()

itemLength_males = 0
numOfItems_males = len(posts_list_male)
for item in posts_list_male:
    itemLength_males = itemLength_males + len(item.split())
    
avg_post_size_males = itemLength_males/numOfItems_males
print("Number of posts by males: ", numOfItems_males)
print("Average number of words per post for males: ", avg_post_size_males)


Number of posts by males:  561
Average number of words per post for males:  40.49910873440285


In [29]:
#Let's calculate the females' average number of words in a post
posts_list_females = female_posts.status_message.tolist()

itemLength_females = 0
numOfItems_females = len(posts_list_females)
for item in posts_list_females:
    itemLength_females = itemLength_females + len(item.split())
    
avg_post_size_females = itemLength_females/numOfItems_females
print("Number of posts by females: ", numOfItems_females)
print("Average number of words per post for females: ", avg_post_size_females)


Number of posts by females:  517
Average number of words per post for females:  36.49323017408124


In [ ]:
#Extract the male DF to a .csv file:
male_posts.to_csv('male_posts.csv', sep=',', index=False)

#Extract the female DF to a .csv file:
female_posts.to_csv('female_posts.csv', sep=',', index=False)


Describing out data:

We scraped posts from 2 different facebook groups and extracted them to 2 .csv files - 1 for each group. Afterwards, we merged the 2 .csv files together to make a single dataset.
Then, we filtered out posts that were too short and posts that only contained a link or a video. facebook only allows us to see the posts content and the post author so had to predict the author's gender based on his/her first name using an external package. After predicting the user's gender, we filtered posts posted by users that we were uncertain about their gender based on the first name.
Finally, we separated male posts and female posts.
Eventually, we ended up with 2 .csv files:
male_posts - which contains 561 posts posted by males, with an average of 40 words per post.
female_posts - which contains 517 posts posted by females, with an average of 36 words per post.

These 2 datasets will be further analyzed when we use the next models.
